In [7]:
%config IPCompleter.use_jedi = False
%pdb off
%load_ext autoreload
%autoreload 3
from pathlib import Path
import pathlib
import numpy as np
import pandas as pd

# required to enable non-blocking interaction:
%gui qt5

## Pho's Custom Libraries:
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path
from pyphocorehelpers.function_helpers import function_attributes

# pyPhoPlaceCellAnalysis:
# NeuroPy (Diba Lab Python Repo) Loading
from neuropy.core.session.Formats.BaseDataSessionFormats import DataSessionFormatRegistryHolder
from neuropy.core.session.Formats.Specific.BapunDataSessionFormat import BapunDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.KDibaOldDataSessionFormat import KDibaOldDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.RachelDataSessionFormat import RachelDataSessionFormat
from neuropy.core.session.Formats.Specific.HiroDataSessionFormat import HiroDataSessionFormatRegisteredClass

## For computation parameters:
from neuropy.utils.result_context import IdentifyingContext
from neuropy.core.session.Formats.BaseDataSessionFormats import find_local_session_paths

# from PendingNotebookCode import _perform_batch_plot, _build_batch_plot_kwargs
# from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_load_session, batch_extended_computations, SessionBatchProgress, batch_programmatic_figures, batch_extended_programmatic_figures
# from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import PipelineSavingScheme
from pyphoplacecellanalysis.General.Pipeline.Stages.Loading import saveData, loadData
import pyphoplacecellanalysis.General.Batch.runBatch
from pyphoplacecellanalysis.General.Batch.runBatch import BatchRun, BatchResultDataframeAccessor, run_diba_batch, BatchComputationProcessOptions, BatchSessionCompletionHandler
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import PipelineSavingScheme
from pyphoplacecellanalysis.General.Model.user_annotations import UserAnnotationsManager

# from pyphocorehelpers.Filesystem.path_helpers import set_posix_windows

Automatic pdb calling has been turned OFF
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
# active_global_batch_result_filename='global_batch_result_2023-07-12.pkl'
# active_global_batch_result_filename='global_batch_result_2023-07-07.pkl'
# active_global_batch_result_filename='global_batch_result_2023-07-07_extra.pkl'
active_global_batch_result_filename='global_batch_result_2023-07-20_lab.pkl'
debug_print = False
enable_neptune = False

if enable_neptune:
    import neptune # for logging progress and results
    from neptune.types import File
    from pyphoplacecellanalysis.General.Batch.NeptuneAiHelpers import set_environment_variables, neptune_output_figures
    neptune_kwargs = {'project':"commander.pho/PhoDibaLongShort2023",
    'api_token':"eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIxOGIxODU2My1lZTNhLTQ2ZWMtOTkzNS02ZTRmNzM5YmNjNjIifQ=="}
    set_environment_variables(neptune_kwargs)

In [9]:
""" 
from pyphoplacecellanalysis.General.Batch.runBatch import main, BatchRun, run_diba_batch, run_specific_batch

"""
known_global_data_root_parent_paths = [Path(r'W:\Data'), Path(r'/media/MAX/Data'), Path(r'/Volumes/MoverNew/data'), Path(r'/home/halechr/turbo/Data'), Path(r'/nfs/turbo/umms-kdiba/Data')]
global_data_root_parent_path = find_first_extant_path(known_global_data_root_parent_paths)
assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"

## NEPTUNE:
if enable_neptune:
	project = neptune.init_project(**neptune_kwargs)
	project["general/global_batch_result_filename"] = active_global_batch_result_filename
	project["general/global_data_root_parent_path"] = global_data_root_parent_path.as_posix()

## TODO: load the batch result initially:

## Build Pickle Path:
global_batch_result_file_path = Path(global_data_root_parent_path).joinpath(active_global_batch_result_filename).resolve() # Use Default

if enable_neptune:
	run = neptune.init_run() # rember to run.stop()
	run['parameters/global_batch_result_file_path'] = global_batch_result_file_path.as_posix()
	# project["general/data_analysis"].upload("data_analysis.ipynb")
	run["dataset/latest"].track_files(f"file://{global_batch_result_file_path}") # "s3://datasets/images"

# try to load an existing batch result:
# with set_posix_windows():
global_batch_run = BatchRun.try_init_from_file(global_data_root_parent_path, active_global_batch_result_filename=active_global_batch_result_filename,
						skip_root_path_conversion=False, debug_print=debug_print) # on_needs_create_callback_fn=run_diba_batch

batch_progress_df = global_batch_run.to_dataframe(expand_context=True, good_only=False) # all
good_only_batch_progress_df = global_batch_run.to_dataframe(expand_context=True, good_only=True)
batch_progress_df.batch_results.build_all_columns()
good_only_batch_progress_df.batch_results.build_all_columns()
if enable_neptune:
	run["dataset/global_batch_run_progress_df"].upload(File.as_html(batch_progress_df)) # "path/to/test_preds.csv"
	run["dataset/global_batch_run_good_only_df"].upload(File.as_html(good_only_batch_progress_df)) # "path/to/test_preds.csv"

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    # display(batch_progress_df)
    display(good_only_batch_progress_df)

Loading loaded session pickle file results : /media/MAX/Data/global_batch_result_2023-07-20_lab.pkl... done.
Failure loading /media/MAX/Data/global_batch_result_2023-07-20_lab.pkl.
creating new batch_run
local_session_names_list: ['2006-6-07_11-26-53', '2006-6-08_14-26-15', '2006-6-09_1-22-43', '2006-6-09_3-23-37', '2006-6-12_15-55-31', '2006-6-13_14-42-6']
local_session_names_list: ['2006-6-07_16-40-19', '2006-6-08_15-46-47', '2006-6-08_21-16-25', '2006-6-09_22-24-40', '2006-6-12_16-53-46', '2006-6-13_15-22-3']
local_session_names_list: ['2006-4-09_17-29-30', '2006-4-10_12-25-50', '2006-4-10_21-2-40', '2006-4-11_15-16-59', '2006-4-12_14-39-31', '2006-4-12_17-53-55', '2006-4-16_15-12-23', '2006-4-17_12-33-47', '2006-4-18_13-6-1', '2006-4-18_15-23-32', '2006-4-19_13-34-40', '2006-4-19_16-48-9', '2006-4-21_10-24-35', '2006-4-25_14-28-51', '2006-4-25_17-17-6', '2006-4-26_13-22-13', '2006-4-27_14-43-12', '2006-4-28_12-17-27', '2006-4-28_16-48-29']
local_session_names_list: ['2006-4-09_16-4

KeyError: 'context_minimal_name'

In [ ]:
df = deepcopy(good_only_batch_progress_df)
df.batch_results._build_minimal_session_identifiers_list()
df


In [10]:
## Get Uniques:
# good_only_batch_progress_df['format_name']
# good_only_batch_progress_df['animal']
# Reset index to work with columns
df.reset_index(inplace=True)

# Extract unique values for each column
unique_format_names = df['format_name'].unique()
unique_animals = df['animal'].unique()
unique_exper_names = df['exper_name'].unique()
unique_session_names = df['session_name'].unique()

# Create mapping to shorthand notation for each column
format_name_mapping = {name: f'f{i}' for i, name in enumerate(unique_format_names)}
animal_mapping = {name: f'a{i}' for i, name in enumerate(unique_animals)}
exper_name_mapping = {name: f'e{i}' for i, name in enumerate(unique_exper_names)}
session_name_mapping = {name: f's{i}' for i, name in enumerate(unique_session_names)}

# df['format_name'].replace(format_name_mapping, inplace=True)
# df['animal'].replace(animal_mapping, inplace=True)
# df['exper_name'].replace(exper_name_mapping, inplace=True)
# df['session_name'].replace(session_name_mapping, inplace=True)




# df['context_minimal_name'] = [f"{animal_mapping[ctx.animal]}{exper_name_mapping[ctx.exper_name]}{session_name_mapping[ctx.session_name]}" for ctx in df['context']] # no [one, two]

## Working one method:
# df['context_minimal_name'] = [f"{animal_mapping[ctx.animal]}{session_name_mapping[ctx.session_name]}" for ctx in df['context']]


# # Set the first four columns as a multi-index
# df.set_index(['format_name', 'animal', 'exper_name', 'session_name'], inplace=True)

# # Sort the DataFrame by the multi-index
# df.sort_index(inplace=True)

df.head()
# Define a function to create a hierarchical mapping
def create_nested_mapping(df, parent_col, child_col):
    mapping = {}
    unique_parents = df[parent_col].unique()
    
    for i, parent in enumerate(unique_parents):
        subset = df[df[parent_col] == parent]
        unique_children = subset[child_col].unique()
        child_mapping = {child: f'{parent[0]}{i}{child_col[0]}{j}' for j, child in enumerate(unique_children)}
        mapping[parent] = child_mapping
    
    return mapping

""" 
{'gor01': {'2006-6-08_14-26-15': 'g0s0',
   '2006-6-12_15-55-31': 'g0s1',
   '2006-6-13_14-42-6': 'g0s2',
   '2006-6-07_16-40-19': 'g0s3',
   '2006-6-08_21-16-25': 'g0s4',
   '2006-6-09_22-24-40': 'g0s5',
   '2006-6-12_16-53-46': 'g0s6',
   '2006-6-13_15-22-3': 'g0s7'},
  'vvp01': {'2006-4-10_12-25-50': 'v1s0',
   '2006-4-09_16-40-54': 'v1s1',
   '2006-4-10_12-58-3': 'v1s2',
   '2006-4-12_15-25-59': 'v1s3',
   '2006-4-16_18-47-52': 'v1s4',
   '2006-4-17_12-52-15': 'v1s5',
   '2006-4-25_13-20-55': 'v1s6',
   '2006-4-28_12-38-13': 'v1s7'},
  'pin01': {'11-02_17-46-44': 'p2s0',
   '11-02_19-28-0': 'p2s1',
   '11-03_12-3-25': 'p2s2',
   '11-09_11-43-50': 'p2s3',
   '11-09_12-15-3': 'p2s4',
   '11-09_21-17-16': 'p2s5',
   '11-09_22-4-5': 'p2s6',
   'fet11-01_12-58-54': 'p2s7',
   'fet11-03_20-28-3': 'p2s8'}})

"""
# Create a mapping for 'session_name' within each 'animal'
# animal_session_mapping = {animal: {session: f'{animal[0]}{i}s{j}' for j, session in enumerate(df[df['animal'] == animal]['session_name'].unique())} for i, animal in enumerate(df['animal'].unique())} # 'g0s0'
animal_session_mapping = {animal: {session: f'{animal_mapping[animal]}s{j}' for j, session in enumerate(df[df['animal'] == animal]['session_name'].unique())} for i, animal in enumerate(df['animal'].unique())} # 'g0s0'

# Replace original values with shorthand notation
for animal, session_mapping in animal_session_mapping.items():
    # df.loc[df['animal'] == animal, 'session_name'] = df.loc[df['animal'] == animal, 'session_name'].replace(session_mapping)
    df.loc[df['animal'] == animal, 'context_minimal_name'] = df.loc[df['animal'] == animal, 'session_name'].replace(session_mapping)

# Drop 'exper_name' column as it's not needed
df.drop(columns='exper_name', inplace=True)

# Display the updated DataFrame and the mappings
# df.head()

df


# # Create a nested mapping for 'exper_name' within each 'animal'
# # animal_exper_mapping = create_nested_mapping(df, 'animal', 'exper_name')
# animal_session_name_mapping = create_nested_mapping(df, 'animal', 'session_name')

# # Create a nested mapping for 'session_name' within each 'exper_name' within each 'animal'
# animal_exper_session_mapping = {animal: create_nested_mapping(df[df['animal'] == animal], 'exper_name', 'session_name') for animal in df['animal'].unique()}

# # Replace original values with shorthand notation
# for animal, exper_mapping in animal_exper_mapping.items():
#     df.loc[df['animal'] == animal, 'exper_name'] = df.loc[df['animal'] == animal, 'exper_name'].replace(exper_mapping)
    
# for animal, exper_session_mapping in animal_exper_session_mapping.items():
#     for exper, session_mapping in exper_session_mapping.items():
#         df.loc[(df['animal'] == animal) & (df['exper_name'] == exper), 'session_name'] = df.loc[(df['animal'] == animal) & (df['exper_name'] == exper), 'session_name'].replace(session_mapping)

# # Display the updated DataFrame and the mappings
# df.head(), animal_exper_mapping, animal_exper_session_mapping


NameError: name 'df' is not defined

In [ ]:
df.head()

In [ ]:
# Export CSV:
# pickle path is: `global_batch_result_file_path`
global_batch_result_CSV_export_file_path = Path(global_data_root_parent_path).joinpath(active_global_batch_result_filename.replace('.pkl', '.csv')).resolve() # Use Default
print(f'global_batch_result_CSV_export_file_path: {global_batch_result_CSV_export_file_path}')
good_only_batch_progress_df.to_csv(global_batch_result_CSV_export_file_path)
            

In [ ]:
important_column_names = ['context', 'basedirs', 'n_long_laps', 'n_long_replays', 'n_short_laps', 'n_short_replays', 'is_ready', 'has_user_replay_annotations']
# file_column_names = ['basedirs', 'global_computation_result_file', 'loaded_session_pickle_file', 'ripple_result_file']
good_only_batch_progress_df[important_column_names]

# change is_ready to show why it isn't ready.
# batch_progress_df['is_ready'] = np.logical_and(batch_progress_df['has_user_replay_annotations'], batch_progress_df['is_ready'])

included_session_contexts = batch_progress_df[np.logical_and(batch_progress_df['has_user_replay_annotations'], batch_progress_df['is_ready'])]['context'].to_numpy().tolist()
included_session_contexts

# Build `global_batch_run` pre-loading results (before execution)

In [14]:
# global_batch_result = loadData('global_batch_result.pkl')
global_batch_run = run_diba_batch(global_data_root_parent_path, execute_all=False, extant_batch_run=global_batch_run, debug_print=False)
# print(f'global_batch_result: {global_batch_run}')
global_batch_run

resusing extant_batch_run: BatchRun(global_data_root_parent_path=PosixPath('/media/MAX/Data'), session_batch_status={IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-07_11-26-53')>: <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>, IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15')>: <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>, IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-09_1-22-43')>: <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>, IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-09_3-23-37')>: <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>, IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-12_15-55-31')>: <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>, IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-13_14-42-6')>: <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>, IdentifyingContext<('kdiba', 'gor01', 'two', '2006-6-07_16-40-19')>: <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>, IdentifyingContext<('kdiba', 'gor01', 'two', '

BatchRun(global_data_root_parent_path=PosixPath('/media/MAX/Data'), session_batch_status={IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-07_11-26-53')>: <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>, IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15')>: <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>, IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-09_1-22-43')>: <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>, IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-09_3-23-37')>: <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>, IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-12_15-55-31')>: <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>, IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-13_14-42-6')>: <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>, IdentifyingContext<('kdiba', 'gor01', 'two', '2006-6-07_16-40-19')>: <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>, IdentifyingContext<('kdiba', 'gor01', 'two', '2006-6-08_15-46-47')>: <Ses

# Run Batch Executions/Computations

In [15]:
## I got it doing the bare-minimum loading and computations, so it should be ready to update the laps and constraint the placefields to those. Then we should be able to set up the replays at the same time.
# finally, we then finish by computing.

included_session_contexts = batch_progress_df[np.logical_and(batch_progress_df['has_user_replay_annotations'], batch_progress_df['is_ready'])]['context'].to_numpy().tolist()
# included_session_contexts

# No Reloading
# result_handler = BatchSessionCompletionHandler(force_reload_all=False,
#                                                 session_computations_options=BatchComputationProcessOptions(should_load=True, should_compute=False, should_save=False),
#                                                 global_computations_options=BatchComputationProcessOptions(should_load=True, should_compute=False, should_save=False),
#                                                 should_perform_figure_generation_to_file=False, saving_mode=PipelineSavingScheme.SKIP_SAVING, force_global_recompute=False)

# Forced Reloading:
result_handler = BatchSessionCompletionHandler(force_reload_all=True,
                                                session_computations_options=BatchComputationProcessOptions(should_load=False, should_compute=True, should_save=True),
                                                global_computations_options=BatchComputationProcessOptions(should_load=False, should_compute=True, should_save=True),
                                                should_perform_figure_generation_to_file=False, saving_mode=PipelineSavingScheme.OVERWRITE_IN_PLACE, force_global_recompute=True)

multiprocessing_kwargs = dict(use_multiprocessing=False, num_processes=1)
# multiprocessing_kwargs = dict(use_multiprocessing=True, num_processes=4)

## Execute with the custom arguments.
active_computation_functions_name_includelist=['_perform_baseline_placefield_computation',
                                        # '_perform_time_dependent_placefield_computation',
                                        '_perform_extended_statistics_computation',
                                        '_perform_position_decoding_computation', 
                                        '_perform_firing_rate_trends_computation',
                                        '_perform_pf_find_ratemap_peaks_computation',
                                        # '_perform_time_dependent_pf_sequential_surprise_computation'
                                        '_perform_two_step_position_decoding_computation',
                                        # '_perform_recursive_latent_placefield_decoding'
                                    ]
# active_computation_functions_name_includelist=['_perform_baseline_placefield_computation']
global_batch_run.execute_all(force_reload=result_handler.force_reload_all, saving_mode=result_handler.saving_mode, skip_extended_batch_computations=True, post_run_callback_fn=result_handler.on_complete_success_execution_session,
                             fail_on_exception=False, included_session_contexts=included_session_contexts,
                                                                                        **{'computation_functions_name_includelist': active_computation_functions_name_includelist,
                                                                                            'active_session_computation_configs': None,
                                                                                            'allow_processing_previously_completed': True}, **multiprocessing_kwargs) # can override `active_session_computation_configs` if we want to set custom ones like only the laps.)

# 4m 39.8s

In [ ]:
# Can override the output filepath like this:
active_global_batch_result_filename='global_batch_result_2023-07-19.pkl'
global_batch_result_file_path = Path(global_data_root_parent_path).joinpath(active_global_batch_result_filename).resolve() # Use Default

In [ ]:
# Save to file:
saveData(global_batch_result_file_path, global_batch_run) # Update the global batch run dictionary

In [ ]:
# list(global_batch_run.session_batch_outputs.keys())

# Somewhere in there there are `InstantaneousSpikeRateGroupsComputation` results to extract
across_sessions_instantaneous_fr_dict = {} # InstantaneousSpikeRateGroupsComputation

for a_ctxt, a_result in global_batch_run.session_batch_outputs.items():
    if a_result is not None:
        a_good_result = a_result.get('across_sessions_batch_results', {}).get('inst_fr_comps', None)
        if a_good_result is not None:
            across_sessions_instantaneous_fr_dict[a_ctxt] = a_good_result
            # print(a_result['across_sessions_batch_results']['inst_fr_comps'])
            
num_sessions = len(across_sessions_instantaneous_fr_dict)
print(f'num_sessions: {num_sessions}')
# When done, `result_handler.across_sessions_instantaneous_fr_dict` is now equivalent to what it would have been before. It can be saved using the normal `.save_across_sessions_data(...)`

In [ ]:
inst_fr_output_filename='across_session_result_long_short_inst_firing_rate_2023-07-19.pkl'
global_batch_result_inst_fr_file_path = Path(global_data_root_parent_path).joinpath(inst_fr_output_filename).resolve() # Use Default
print(f'global_batch_result_inst_fr_file_path: {global_batch_result_inst_fr_file_path}')
# Save the all sessions instantaneous firing rate dict to the path:
saveData(global_batch_result_inst_fr_file_path, across_sessions_instantaneous_fr_dict)

In [ ]:
## Save the instantaneous firing rate results dict: (# Dict[IdentifyingContext] = InstantaneousSpikeRateGroupsComputation)
#TODO 2023-07-12 10:12: - [ ] New save way after we save out current result and reload
result_handler.save_across_sessions_data(global_data_root_parent_path=global_data_root_parent_path, inst_fr_output_filename='across_session_result_long_short_inst_firing_rate_2023-07-19.pkl')
num_sessions = len(result_handler.across_sessions_instantaneous_fr_dict)
print(f'num_sessions: {num_sessions}')

# 2023-07-14 - Load Saved across-sessions-data and testing Batch-computed inst_firing_rates:

In [ ]:
from neuropy.utils.matplotlib_helpers import matplotlib_configuration_update
from pyphoplacecellanalysis.General.Batch.PhoDiba2023Paper import PaperFigureTwo, InstantaneousSpikeRateGroupsComputation
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.SpikeAnalysis import SpikeRateTrends

## Load the saved across-session results:
# inst_fr_output_filename = 'long_short_inst_firing_rate_result_handlers_2023-07-12.pkl'
# inst_fr_output_filename = 'across_session_result_long_short_inst_firing_rate.pkl'
inst_fr_output_filename='across_session_result_long_short_inst_firing_rate_2023-07-19.pkl'
across_session_inst_fr_computation, across_sessions_instantaneous_fr_dict, across_sessions_instantaneous_frs_list = BatchSessionCompletionHandler.load_across_sessions_data(global_data_root_parent_path=global_data_root_parent_path, inst_fr_output_filename=inst_fr_output_filename)
# across_sessions_instantaneous_fr_dict = loadData(global_batch_result_inst_fr_file_path)
num_sessions = len(across_sessions_instantaneous_fr_dict)
print(f'num_sessions: {num_sessions}')

In [ ]:
## Aggregate across all of the sessions to build a new combined `InstantaneousSpikeRateGroupsComputation`, which can be used to plot the "PaperFigureTwo", bar plots for many sessions.
global_multi_session_context = IdentifyingContext(format_name='kdiba', num_sessions=num_sessions) # some global context across all of the sessions, not sure what to put here.

# To correctly aggregate results across sessions, it only makes sense to combine entries at the `.cell_agg_inst_fr_list` variable and lower (as the number of cells can be added across sessions, treated as unique for each session).

## Display the aggregate across sessions:
_out_fig_2 = PaperFigureTwo(instantaneous_time_bin_size_seconds=0.01) # WARNING: we didn't save this info
_out_fig_2.computation_result = across_session_inst_fr_computation
_out_fig_2.active_identifying_session_ctx = across_session_inst_fr_computation.active_identifying_session_ctx
# Set callback, the only self-specific property
# _out_fig_2._pipeline_file_callback_fn = curr_active_pipeline.output_figure # lambda args, kwargs: self.write_to_file(args, kwargs, curr_active_pipeline)

In [ ]:
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import FigureOutputManager, FigureOutputLocation, ContextToPathMode

registered_output_files = {}

def output_figure(final_context: IdentifyingContext, fig, write_vector_format:bool=False, write_png:bool=True, debug_print=True):
    """ outputs the figure using the provided context. """
    from pyphoplacecellanalysis.General.Mixins.ExportHelpers import build_and_write_to_file
    def register_output_file(output_path, output_metadata=None):
        """ registers a new output file for the pipeline """
        print(f'register_output_file(output_path: {output_path}, ...)')
        registered_output_files[output_path] = output_metadata or {}

    fig_out_man = FigureOutputManager(figure_output_location=FigureOutputLocation.DAILY_PROGRAMMATIC_OUTPUT_FOLDER, context_to_path_mode=ContextToPathMode.HIERARCHY_UNIQUE)
    active_out_figure_paths = build_and_write_to_file(fig, final_context, fig_out_man, write_vector_format=write_vector_format, write_png=write_png, register_output_file_fn=register_output_file)
    return active_out_figure_paths, final_context


# Set callback, the only self-specific property
_out_fig_2._pipeline_file_callback_fn = output_figure

In [ ]:
# Showing
restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
# Perform interactive Matplotlib operations with 'Qt5Agg' backend
_fig_2_theta_out, _fig_2_replay_out = _out_fig_2.display(active_context=global_multi_session_context, title_modifier_fn=lambda original_title: f"{original_title} ({num_sessions} sessions)", save_figure=True)
	
_out_fig_2.perform_save()

## Single Session testing:


In [ ]:
_test_out = global_batch_run.execute_session(session_context=curr_sess_context, force_reload=True, skip_extended_batch_computations=True, computation_functions_name_includelist =['_perform_baseline_placefield_computation'], active_session_computation_configs=None) # can override `active_session_computation_configs` if we want to set custom ones like only the laps.)
_test_out

# global_batch_run.execute_session(session_context=curr_sess_context, force_reload=True, skip_extended_batch_computations=True, **{'computation_functions_name_includelist': ['_perform_baseline_placefield_computation'], 'active_session_computation_configs': None}) # can override `active_session_computation_configs` if we want to set custom ones like only the laps.)

# 23.5s

In [ ]:

full_good_dirs = [k for k, v in global_batch_run.session_batch_errors.items() if v is None]
bad_dirs = [k for k, v in global_batch_run.session_batch_errors.items() if v is not None]
full_good_dirs
bad_dirs

In [ ]:
global_batch_run.session_batch_status

In [ ]:
global_batch_run.session_batch_status
global_batch_run.session_batch_errors

# Get good sessions for use in the specific session processing notebook:

In [ ]:
batch_progress_df = global_batch_run.to_dataframe(expand_context=True, good_only=False) # all
good_only_batch_progress_df = global_batch_run.to_dataframe(expand_context=True, good_only=True)
good_only_batch_progress_df

In [ ]:
## Get the list of sessions that are completely ready to process:
full_good_ready_to_process_sessions = list(good_only_batch_progress_df['context'].to_numpy())
full_good_ready_to_process_sessions
# Get good sessions for use in the specific session processing notebook:

In [ ]:
run["good_sessions_list"].extend(full_good_ready_to_process_sessions)

In [ ]:
run.stop()
project.stop()

In [ ]:

print(",\n".join([ctx.get_initialization_code_string() for ctx in full_good_ready_to_process_sessions])) # List definitions

# [IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15'),
# IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43'),
# IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31'),
# IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-13_14-42-6'),
# IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19'),
# IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46'),
# IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30')]

In [ ]:
print("\ncurr_context = ".join([ctx.get_initialization_code_string() for ctx in full_good_ready_to_process_sessions])) # Line definitions

# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15')
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43')
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31')
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-13_14-42-6')
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19')
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30')

In [ ]:
good_only_batch_progress_df

In [ ]:
from datetime import datetime

# datetime object containing current date and time
save_time = datetime.now()
 
print("save_time =", save_time)

# dd/mm/YY H:M:S
dt_string = save_time.strftime("%Y-%m-%d_%I-%M%p")
print("date and time =", dt_string)

In [ ]:
## Get output file paths:
completed_pipeline_filename = 'loadedSessPickle.pkl'
completed_global_computations_filename = 'outputs/global_computation_results.pkl'

full_good_ready_to_process_session_paths = list(good_only_batch_progress_df['basedirs'].to_numpy())
session_paths_output_folders = [sess_path.joinpath('outputs').resolve() for sess_path in full_good_ready_to_process_session_paths]



completed_pipeline_file_paths = [sess_path.joinpath(completed_pipeline_filename).resolve() for sess_path in full_good_ready_to_process_session_paths]
completed_global_computations_file_paths = [sess_path.joinpath(completed_global_computations_filename).resolve() for sess_path in full_good_ready_to_process_session_paths]
completed_global_computations_file_paths